In [27]:
import json
        
data_list = []
with open("../data/raw/batchoutput_1000_gpt-4o-mini.jsonl", "r") as f:
    for line in f:
        data_list.append( json.loads(line) )
        
data_list[0]

{'id': 'batch_req_674c8731ddb48190b32f03fc471d0898',
 'custom_id': '1427_10_17_2',
 'response': {'status_code': 200,
  'request_id': '8811c4d08fd3689a7770005741879986',
  'body': {'id': 'chatcmpl-AZgAhuyuOEOboy5h6dKyESG8PaAqP',
   'object': 'chat.completion',
   'created': 1733066951,
   'model': 'gpt-4o-mini-2024-07-18',
   'choices': [{'index': 0,
     'message': {'role': 'assistant',
      'content': '{"translated": {"The Right Minister of State, Kim Hyodeung and others submitted a petition stating: It is my humble opinion that all actions taken should consider the abundance or scarcity of the year. In recent years, we have faced alternating floods and droughts, leading to poor harvests. This year, during the farming month, the drought is particularly severe. Your Highness has shown great concern for the people, implementing measures to alleviate famine and care for the populace, which is commendable. Currently, the agricultural conditions in the provinces of Chungcheong, Gyeongsang

In [ ]:
import pandas as pd
df_sentences_filtered = pd.read_csv("../data/raw/sentencePair_세종실록_filtered.csv")
df_sentences_filtered_sample = df_sentences_filtered.sample(1000, random_state=42)
df_sentences_filtered_sample

In [54]:
import pandas as pd
import re

# 데이터프레임 로드 및 샘플 추출
df_sentences_filtered = pd.read_csv("../data/raw/sentencePair_세종실록_filtered.csv")
df_sentences_filtered_sample = df_sentences_filtered.sample(1000, random_state=42).reset_index(drop=True)

# JSONL 파일에서 각 줄을 읽어서 파싱
with open('../data/raw/batchoutput_1000_gpt-4o.jsonl', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# 데이터프레임의 행 수와 파일의 줄 수가 일치하는지 확인
if len(lines) != len(df_sentences_filtered_sample):
    print(f"경고: JSONL 파일의 줄 수({len(lines)})와 데이터프레임의 행 수({len(df_sentences_filtered_sample)})가 일치하지 않습니다.")
    # 필요에 따라 이 상황을 처리하세요.

# 번역된 내용을 저장할 리스트 초기화
translated_contents = []

def extract_translated_content_custom(content_str):
    # "translated": 이후의 내용을 추출
    start_index = content_str.find('"translated":')
    if start_index == -1:
        return content_str.strip()  # "translated" 키가 없으면 원본 반환

    # "translated": 다음부터 파싱 시작
    index = start_index + len('"translated":')
    # 공백 제거
    while index < len(content_str) and content_str[index] in ' \t\n\r':
        index += 1

    # 중괄호로 시작하는지 확인
    if index < len(content_str) and content_str[index] == '{':
        # 중괄호의 짝을 맞추어 내용 추출
        stack = ['{']
        index += 1
        start_content = index
        while index < len(content_str) and stack:
            char = content_str[index]
            if char == '{':
                stack.append('{')
            elif char == '}':
                stack.pop()
            index += 1
        end_content = index - 1  # 마지막 '}'의 위치
        translated_text = content_str[start_content:end_content].strip()
        return translated_text
    elif index < len(content_str) and content_str[index] == '"':
        # 따옴표로 시작하면 따옴표 짝을 맞추어 내용 추출
        index += 1
        start_content = index
        while index < len(content_str):
            char = content_str[index]
            if char == '"' and content_str[index - 1] != '\\':
                break
            index += 1
        translated_text = content_str[start_content:index].replace('\\"', '"').replace('\\\\', '\\').strip()
        return translated_text
    else:
        # 그 외의 경우는 다음 콤마 또는 끝까지 내용을 추출
        start_content = index
        while index < len(content_str) and content_str[index] != ',':
            index += 1
        translated_text = content_str[start_content:index].strip()
        return translated_text

# 각 줄에 대해 파싱 및 번역된 내용 추출
for line_number, line in enumerate(lines):
    line = line.strip()
    if not line:
        translated_contents.append('')  # 빈 줄 처리
        continue

    # 데이터의 특정 구조를 고려하여 'content' 필드 추출
    try:
        data = json.loads(line)
        response = data.get('response', {})
        body = response.get('body', {})
        choices = body.get('choices', [])
        if choices:
            content = choices[0].get('message', {}).get('content', '')
        else:
            content = ''
    except json.JSONDecodeError as e:
        print(f"라인 {line_number + 1}: JSONDecodeError - {e}")
        translated_contents.append('')  # 오류 시 빈 문자열 추가
        continue

    # 번역된 콘텐츠 추출
    translated_content = extract_translated_content_custom(content)
    translated_contents.append(translated_content)

# 추출한 번역된 내용을 데이터프레임의 새로운 열로 추가
df_sentences_filtered_sample['predictions_Eng'] = translated_contents

# 결과 확인
print(df_sentences_filtered_sample.head())

# 필요하면 결과를 CSV 파일로 저장
df_sentences_filtered_sample.to_csv('../data/processed/batchoutput_1000_gpt-4o.csv', index=False)

   Unnamed: 0 annals  year month  day  sentence_index  \
0        4489   세종실록  1427    10   17               2   
1       29311   세종실록  1431     6   30               0   
2       25117   세종실록  1422    12    5               4   
3       16376   세종실록  1433     5    6               2   
4        3213   세종실록  1419    10    3               2   

                                               hanja  \
0  ○右司諫金孝貞等上疏曰: 竊謂凡所施爲, 視歲豐歉。 比年以來, 水旱相仍, 禾稼不稔, 而...   
1                                        ○壬戌/親傳朔祭香祝。   
2  ○全羅道觀察使啓: "有老病父母生員等爲本鄕學長, 敎訓生徒, 兼侍親病, 今以無圓點, 未...   
3  ○內傳于咸吉道監司曰: "遣金思允等四五名于新地, 開諭招安。" 仍令曰: "若其人恐被避役...   
4  ○慶尙道右道兵馬節制使啓: "對馬島已遣人納款, 且今秋耕拾栗爲緊。 其屯守陸地軍人等, 姑...   

                                              korean  \
0  우사간 김효정 등이 상소하기를, "그윽이 생각하옵건대, 모든 행사는 시절의 흉풍을 ...   
1                              삭제에 쓸 향과 축문을 친히 전하였다.   
2  전라도 관찰사가 계하기를, "늙고 병든 부모가 있는 생원들을 본향의 학장으로 삼아 ...   
3  함길도 감사에게 내전하기를, "김사윤 등 4, 5명을 새 땅으로 보내어 초안할 것을...   
4  경상우도 병마 절제사가 계하기를, "대마도에서 이미 사람을 보내 납